['CSCO.csv', 'ISRG.csv', 'PDD.csv', 'VRTX.csv', 'GILD.csv', 'FOX.csv', 'SWKS.csv', 'MCHP.csv', 'CDNS.csv', 'CHTR.csv', 'KDP.csv', 'WBA.csv', 'ADI.csv', 'ADBE.csv', 'TCOM.csv', 'CPRT.csv', 'PEP.csv', 'QUALCOMM.csv', 'COST.csv', 'MDLZ.csv', 'BKNG.csv', 'XEL.csv', 'WDAY.csv', 'CERN.csv', 'FAST.csv', 'KHC.csv', 'CHKP.csv', 'EBAY.csv', 'SBUX.csv', 'INTU.csv', 'MNST.csv', 'AMZN.csv', 'ANSYS.csv', 'INTC.csv', 'DOCU.csv', 'DXCM.csv', 'TXN.csv', 'VRSK.csv', 'MELI.csv', 'ADSK.csv', 'PCAR.csv', 'PYPL.csv', 'BIDU.csv', 'MSFT.csv', 'ANSS.csv', 'CTAS.csv', 'OKTA.csv', 'NTES.csv', 'VRSN.csv', 'AMAT.csv', 'ATVI.csv', 'EA.csv', 'AMD.csv', 'KLAC.csv', 'NXPI.csv', 'MTCH.csv', 'BIIB.csv', 'NVDA.csv', 'IDXX.csv', 'EXC.csv', 'INCY.csv', 'MRVL.csv', 'CMCSA.csv', 'SIRI.csv', 'MAR.csv', 'FB.csv', 'SGEN.csv', 'LULU.csv', 'ALGN.csv', 'NFLX.csv', 'PTON.csv', 'MRNA.csv', 'TSLA.csv', 'GOOGL.csv', 'PAYX.csv', 'ILMN.csv', 'XLNX.csv', 'FOXA.csv', 'JD.csv', 'ORLY.csv', 'SNPS.csv', 'GOOG.csv', 'AVGO.csv', 'REGN.csv', 'A

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.149240                -0.106422  0.023536  0.061529   
2015-01-05     -0.219815                -0.193255  0.030489  0.077491   
2015-01-06     -0.147910                -0.104822  0.049468  0.124367   
2015-01-07     -0.113289                -0.061344  0.028478  0.073163   
2015-01-08     -0.118616                -0.068356  0.042668  0.109391   
...                  ...                      ...       ...       ...   
2019-12-24     -0.196106                -0.133242  0.006638  0.038026   
2019-12-26     -0.135745                -0.096523  0.010944  0.059963   
2019-12-27     -0.158960                -0.110712  0.010907  0.059674   
2019-12-30     -0.174438                -0.120194  0.012047  0.065213   
2019-12-31     -0.089312                -0.067982  0.014988  0.080698   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.002242                -0.246102  0.019359  0.040016   
2015-01-05     -0.037654                -0.302581  0.047681  0.094603   
2015-01-06      0.047030                -0.182177  0.030980  0.062563   
2015-01-07      0.033935                -0.201285  0.041724  0.084277   
2015-01-08      0.072200                -0.147072  0.045028  0.092651   
...                  ...                      ...       ...       ...   
2019-12-24      0.040661                -0.213218  0.009806  0.071569   
2019-12-26      0.007643                -0.226876  0.011750  0.084752   
2019-12-27      0.016968                -0.223026  0.015407  0.109687   
2019-12-30     -0.080862                -0.263496  0.013481  0.095484   
2019-12-31      0.016815                -0.223101  0.016118  0.113256   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2018-10-01     -0.017394                 0.066205  0.057188  0.021353   
2018-10-02     -0.180844                -0.412092  0.048229  0.016514   
2018-10-03     -0.054563                -0.043729  0.016594  0.006759   
2018-10-04     -0.254706                -0.649848  0.105133  0.029933   
2018-10-05     -0.127949                -0.288202  0.166117  0.044931   
...                  ...                      ...       ...       ...   
2019-12-24     -0.076960                -0.120079  0.011000  0.009751   
2019-12-26     -0.058852                -0.082387  0.020750  0.014625   
2019-12-27     -0.052180                -0.069393  0.045005  0.026765   
2019-12-30     -0.136050                -0.240015  0.032855  0.020120   
2019-12-31     -0.099833                -0.167730  0.013669  0.010911   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.359802                -0.182020  0.033946  0.046880   
2015-01-05      0.217861                -0.288478  0.039782  0.054028   
2015-01-06      0.181194                -0.316345  0.054183  0.071356   
2015-01-07      0.337982                -0.197880  0.028868  0.039086   
2015-01-08      0.338207                -0.199271  0.032143  0.044714   
...                  ...                      ...       ...       ...   
2019-12-24      0.276797                -0.249534  0.007111  0.017600   
2019-12-26      0.256777                -0.257727  0.016494  0.040764   
2019-12-27      0.279946                -0.248252  0.021406  0.053079   
2019-12-30      0.218760                -0.273225  0.018323  0.045022   
2019-12-31      0.263300                -0.255063  0.024525  0.060257   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.342504                -0.007273  0.027099  0.128274   
2015-01-05      0.444436                 0.046789  0.060456  0.291842   
2015-01-06      0.359908                 0.001057  0.049111  0.239179   
2015-01-07      0.440294                 0.042319  0.035956  0.178397   
2015-01-08      0.522334                 0.082540  0.041509  0.211785   
...                  ...                      ...       ...       ...   
2019-12-24      0.285802                -0.038025  0.004317  0.016399   
2019-12-26      0.257418                -0.056852  0.007746  0.029280   
2019-12-27      0.247013                -0.063891  0.009803  0.036808   
2019-12-30      0.192144                -0.100897  0.012878  0.047607   
2019-12-31      0.293369                -0.032919  0.015338  0.056747   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.015490                -0.383493  0.069990  0.055570   
2015-01-05     -0.120187                -0.442235  0.071814  0.056093   
2015-01-06     -0.184749                -0.479590  0.116551  0.088133   
2015-01-07      0.015917                -0.364218  0.071119  0.054664   
2015-01-08      0.136320                -0.295271  0.092650  0.074444   
...                  ...                      ...       ...       ...   
2019-12-24      0.082382                -0.361170  0.026911  0.038318   
2019-12-26     -0.043458                -0.401166  0.036161  0.051540   
2019-12-27     -0.050345                -0.403315  0.034725  0.049496   
2019-12-30     -0.129855                -0.428093  0.031375  0.044261   
2019-12-31     -0.060362                -0.406467  0.028763  0.040525   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.067601                -0.130086  0.021744  0.019310   
2015-01-05      0.015782                -0.201596  0.022135  0.019219   
2015-01-06      0.026010                -0.189326  0.031033  0.026423   
2015-01-07      0.100327                -0.084150  0.023054  0.019739   
2015-01-08      0.178058                 0.026794  0.033558  0.029649   
...                  ...                      ...       ...       ...   
2019-12-24      0.089291                -0.104851  0.010133  0.023508   
2019-12-26      0.150915                -0.072354  0.020555  0.048087   
2019-12-27      0.049729                -0.125567  0.014200  0.033078   
2019-12-30      0.046686                -0.127241  0.025035  0.058048   
2019-12-31      0.063424                -0.118500  0.034172  0.079028   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.067868                 0.178962  0.061309  0.030166   
2015-01-05      0.033033                 0.132480  0.158515  0.076213   
2015-01-06      0.025826                 0.121069  0.075169  0.035178   
2015-01-07      0.145946                 0.286795  0.050905  0.024502   
2015-01-08      0.107507                 0.232178  0.038430  0.018688   
...                  ...                      ...       ...       ...   
2019-12-24      0.025826                 0.180147  0.007629  0.013884   
2019-12-26      0.132733                 0.218429  0.011683  0.021385   
2019-12-27      0.113513                 0.211461  0.015740  0.028909   
2019-12-30      0.048648                 0.188321  0.014780  0.027031   
2019-12-31      0.009009                 0.174087  0.026892  0.048738   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.206583                -0.003151  0.020106  0.012056   
2015-01-05      0.099852                -0.245023  0.034754  0.020097   
2015-01-06      0.132206                -0.178229  0.040015  0.022535   
2015-01-07      0.219041                 0.026381  0.031655  0.017882   
2015-01-08      0.324471                 0.279918  0.032992  0.019368   
...                  ...                      ...       ...       ...   
2019-12-24      0.212719                 0.006888  0.013074  0.022651   
2019-12-26      0.300670                 0.075869  0.021228  0.037150   
2019-12-27      0.172183                -0.024607  0.014309  0.024936   
2019-12-30      0.198773                -0.003998  0.019807  0.034474   
2019-12-31      0.247304                 0.033880  0.021347  0.037307   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.068001                -0.157607  0.014089  0.005741   
2015-01-05     -0.072871                -0.167669  0.021018  0.008066   
2015-01-06     -0.061740                -0.144661  0.023066  0.008772   
2015-01-07     -0.029037                -0.077028  0.016236  0.006560   
2015-01-08     -0.006772                -0.032709  0.027419  0.010657   
...                  ...                      ...       ...       ...   
2019-12-24     -0.103665                -0.180431  0.005824  0.009044   
2019-12-26     -0.024368                -0.121345  0.010806  0.013801   
2019-12-27     -0.060412                -0.148296  0.008389  0.011529   
2019-12-30     -0.058009                -0.146515  0.014409  0.017231   
2019-12-31     -0.045994                -0.137612  0.019829  0.022416   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.254873                -0.068722  0.046524  0.060722   
2015-01-05      0.093896                -0.176949  0.056386  0.072059   
2015-01-06      0.303166                -0.035992  0.057738  0.073965   
2015-01-07      0.516146                 0.109610  0.067137  0.088141   
2015-01-08      0.397276                 0.026317  0.048253  0.064248   
...                  ...                      ...       ...       ...   
2019-12-24      0.249248                -0.075880  0.017238  0.019481   
2019-12-26      0.355609                 0.007493  0.032228  0.036468   
2019-12-27      0.296213                -0.039237  0.043065  0.048716   
2019-12-30      0.264443                -0.063910  0.038510  0.043518   
2019-12-31      0.286543                -0.046755  0.038993  0.044097   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.152218                -0.037614  0.047549  0.038510   
2015-01-05      0.088962                -0.129786  0.051946  0.041296   
2015-01-06      0.072381                -0.156068  0.061401  0.047646   
2015-01-07      0.185381                 0.012862  0.061959  0.048585   
2015-01-08      0.209331                 0.048339  0.067434  0.053801   
...                  ...                      ...       ...       ...   
2019-12-24      0.188964                -0.016467  0.017438  0.034406   
2019-12-26      0.151680                -0.038990  0.032749  0.064374   
2019-12-27      0.131657                -0.051028  0.048908  0.095774   
2019-12-30      0.110252                -0.063954  0.039767  0.077541   
2019-12-31      0.149608                -0.040241  0.034584  0.067460   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.017505                -0.048005  0.013133  0.041327   
2015-01-05     -0.017505                -0.048083  0.013441  0.042069   
2015-01-06     -0.039312                -0.096698  0.016246  0.049662   
2015-01-07      0.001300                -0.005777  0.013299  0.041128   
2015-01-08      0.025908                 0.049446  0.014700  0.046536   
...                  ...                      ...       ...       ...   
2019-12-24      0.003501                -0.025411  0.005822  0.085490   
2019-12-26      0.020906                -0.016966  0.005758  0.084984   
2019-12-27     -0.018506                -0.036101  0.006753  0.098933   
2019-12-30     -0.059318                -0.055872  0.010554  0.151455   
2019-12-31      0.019106                -0.017784  0.008819  0.127717   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.132024                -0.418580  0.052179  0.048189   
2015-01-05     -0.156398                -0.443852  0.042070  0.038342   
2015-01-06     -0.073799                -0.358034  0.056656  0.052236   
2015-01-07     -0.064997                -0.349467  0.070533  0.066000   
2015-01-08     -0.067705                -0.352991  0.052840  0.050221   
...                  ...                      ...       ...       ...   
2019-12-24     -0.138795                -0.416457  0.005348  0.007828   
2019-12-26     -0.099526                -0.389972  0.008024  0.011613   
2019-12-27     -0.107651                -0.395482  0.014935  0.021353   
2019-12-30     -0.168585                -0.436564  0.018458  0.026000   
2019-12-31     -0.202437                -0.460107  0.019860  0.027410   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.013473                 0.157912  0.017997  0.007563   
2015-01-05     -0.035929                 0.109459  0.015477  0.006439   
2015-01-06     -0.038424                 0.103640  0.011293  0.004646   
2015-01-07     -0.012476                 0.160294  0.010242  0.004226   
2015-01-08      0.000499                 0.188753  0.014702  0.006129   
...                  ...                      ...       ...       ...   
2019-12-24      0.085828                 0.193758  0.007861  0.016418   
2019-12-26     -0.006987                 0.152965  0.011262  0.023550   
2019-12-27     -0.037924                 0.139552  0.012719  0.026540   
2019-12-30     -0.013972                 0.149936  0.017351  0.036220   
2019-12-31     -0.009980                 0.151668  0.016881  0.035270   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.014660                -0.069391  0.076809  0.094593   
2015-01-05     -0.049498                -0.110466  0.139528  0.170543   
2015-01-06     -0.049497                -0.110838  0.134199  0.162787   
2015-01-07      0.153022                 0.131138  0.141376  0.176507   
2015-01-08      0.095162                 0.058404  0.154488  0.196384   
...                  ...                      ...       ...       ...   
2019-12-24     -0.017837                -0.068245  0.026441  0.054687   
2019-12-26     -0.021258                -0.070654  0.063573  0.131293   
2019-12-27      0.050572                -0.020179  0.055326  0.114972   
2019-12-30     -0.057515                -0.095936  0.065522  0.135438   
2019-12-31     -0.017153                -0.067777  0.069957  0.144456   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.152787                 0.091423  0.025113  0.046491   
2015-01-05     -0.205796                 0.042998  0.030005  0.054645   
2015-01-06     -0.081028                 0.157667  0.055332  0.100923   
2015-01-07     -0.058668                 0.177254  0.037464  0.069983   
2015-01-08     -0.103026                 0.136105  0.050969  0.095511   
...                  ...                      ...       ...       ...   
2019-12-24     -0.110107                 0.110661  0.008765  0.040482   
2019-12-26     -0.086381                 0.119810  0.018605  0.082545   
2019-12-27     -0.214005                 0.070478  0.024551  0.107246   
2019-12-30     -0.105605                 0.112367  0.021022  0.092640   
2019-12-31     -0.197641                 0.076749  0.026518  0.115494   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.182568                 0.194909  0.072817  0.157398   
2015-01-05      0.008560                -0.027066  0.091696  0.194192   
2015-01-06      0.060249                 0.037665  0.072440  0.153136   
2015-01-07      0.297998                 0.345531  0.099566  0.215712   
2015-01-08      0.184291                 0.194909  0.076906  0.169687   
...                  ...                      ...       ...       ...   
2019-12-24      0.152353                 0.131312  0.014862  0.059522   
2019-12-26      0.076914                 0.072653  0.031947  0.118125   
2019-12-27      0.184415                 0.156144  0.038497  0.141779   
2019-12-30      0.104265                 0.093850  0.027395  0.103303   
2019-12-31      0.118485                 0.104814  0.046491  0.169755   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.124220                 0.068374  0.029895  0.106231   
2015-01-05     -0.280802                -0.033630  0.064201  0.210810   
2015-01-06     -0.190023                 0.023931  0.066112  0.213385   
2015-01-07     -0.165588                 0.040105  0.058106  0.186605   
2015-01-08     -0.085591                 0.095512  0.063853  0.206892   
...                  ...                      ...       ...       ...   
2019-12-24     -0.090255                 0.079018  0.003818  0.036278   
2019-12-26     -0.061797                 0.089289  0.009217  0.068820   
2019-12-27     -0.102692                 0.074306  0.015382  0.105987   
2019-12-30     -0.193571                 0.041645  0.011881  0.084274   
2019-12-31     -0.130947                 0.064123  0.012021  0.085094   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.149370                 0.239312  0.102009  0.111074   
2015-01-05      0.080873                 0.183132  0.125039  0.134606   
2015-01-06      0.148244                 0.238488  0.191132  0.206850   
2015-01-07      0.161719                 0.249494  0.114021  0.124463   
2015-01-08      0.174071                 0.259336  0.101280  0.111838   
...                  ...                      ...       ...       ...   
2019-12-24      0.159260                 0.233814  0.038507  0.085804   
2019-12-26      0.117668                 0.217125  0.053316  0.118670   
2019-12-27      0.170482                 0.238273  0.077715  0.173884   
2019-12-30      0.149356                 0.229765  0.069123  0.155077   
2019-12-31      0.153318                 0.231315  0.085993  0.193533   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.169578                -0.150451  0.065989  0.045734   
2015-01-05     -0.136690                -0.094893  0.064314  0.044368   
2015-01-06     -0.144501                -0.108595  0.063993  0.043811   
2015-01-07     -0.123124                -0.071520  0.043255  0.029928   
2015-01-08      0.020761                 0.180347  0.085955  0.061047   
...                  ...                      ...       ...       ...   
2019-12-24     -0.096403                -0.046488  0.011274  0.020281   
2019-12-26     -0.154368                -0.095005  0.039220  0.059042   
2019-12-27     -0.100102                -0.049566  0.044566  0.066682   
2019-12-30     -0.202878                -0.135692  0.046445  0.068437   
2019-12-31     -0.095169                -0.045226  0.030944  0.047258   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.128852                 0.249179  0.005388  0.054404   
2015-01-05     -0.065613                 0.197313  0.005593  0.055720   
2015-01-06     -0.032411                 0.205787  0.005266  0.051938   
2015-01-07      0.176283                 0.262389  0.004445  0.044303   
2015-01-08      0.293281                 0.293646  0.004316  0.043959   
...                  ...                      ...       ...       ...   
2019-12-24      0.046516                 0.227923  0.001283  0.014675   
2019-12-26      0.056020                 0.230146  0.002975  0.033982   
2019-12-27      0.111775                 0.243262  0.003309  0.037933   
2019-12-30      0.043273                 0.227165  0.005818  0.066543   
2019-12-31      0.114961                 0.243999  0.005055  0.058045   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.207711                -0.002013  0.035360  0.040200   
2015-01-05     -0.271455                -0.074623  0.050399  0.056317   
2015-01-06     -0.270473                -0.075059  0.047138  0.051769   
2015-01-07     -0.221439                -0.018876  0.056594  0.061747   
2015-01-08     -0.091013                 0.135573  0.041966  0.046824   
...                  ...                      ...       ...       ...   
2019-12-24     -0.173991                 0.027245  0.010679  0.021785   
2019-12-26     -0.207798                 0.005778  0.024173  0.049221   
2019-12-27     -0.183004                 0.021514  0.022907  0.046695   
2019-12-30     -0.198782                 0.011493  0.018360  0.037395   
2019-12-31     -0.192020                 0.015785  0.028231  0.057500   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-09-09      0.050841                 0.060440  0.016061  0.029260   
2015-09-10      0.204233                 0.169118  0.021144  0.038655   
2015-09-11      0.334902                 0.264160  0.010187  0.019044   
2015-09-14      0.269163                 0.215333  0.015994  0.030464   
2015-09-15      0.273218                 0.217704  0.019460  0.037630   
...                  ...                      ...       ...       ...   
2019-12-24      0.210813                 0.177715  0.006078  0.000370   
2019-12-26      0.171395                 0.123088  0.023660  0.017129   
2019-12-27      0.204084                 0.168362  0.024641  0.018062   
2019-12-30      0.213698                 0.181833  0.022906  0.016475   
2019-12-31      0.245426                 0.226129  0.039368  0.032900   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.175806                -0.093587  0.008447  0.019275   
2015-01-05     -0.193819                -0.108186  0.011097  0.024308   
2015-01-06     -0.173157                -0.091463  0.027174  0.055472   
2015-01-07     -0.136071                -0.061225  0.011637  0.025545   
2015-01-08     -0.067196                -0.005747  0.025868  0.054695   
...                  ...                      ...       ...       ...   
2019-12-24     -0.178985                -0.093996  0.001643  0.008571   
2019-12-26     -0.144018                -0.074085  0.006903  0.023279   
2019-12-27     -0.184283                -0.096986  0.007052  0.023636   
2019-12-30     -0.176336                -0.092487  0.010383  0.032868   
2019-12-31     -0.208124                -0.110583  0.007846  0.025651   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.141848                -0.380644  0.020661  0.044901   
2015-01-05     -0.190146                -0.410409  0.027043  0.056466   
2015-01-06     -0.211707                -0.424016  0.036134  0.072528   
2015-01-07     -0.157372                -0.390320  0.034492  0.069377   
2015-01-08     -0.021964                -0.304957  0.038566  0.078997   
...                  ...                      ...       ...       ...   
2019-12-24     -0.149610                -0.385434  0.000000  0.009752   
2019-12-26     -0.141290                -0.382135  0.003393  0.019729   
2019-12-27     -0.187052                -0.400265  0.007780  0.032448   
2019-12-30     -0.205775                -0.407792  0.007656  0.031838   
2019-12-31     -0.085124                -0.359570  0.009223  0.036708   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.124300                 0.180421  0.009216  0.081520   
2015-01-05      0.080234                 0.130186  0.017328  0.135111   
2015-01-06      0.122445                 0.177409  0.010549  0.088290   
2015-01-07      0.243042                 0.317764  0.014090  0.114938   
2015-01-08      0.213356                 0.281442  0.019977  0.158128   
...                  ...                      ...       ...       ...   
2019-12-24      0.181959                 0.226369  0.000589  0.051830   
2019-12-26      0.108040                 0.190979  0.004578  0.116863   
2019-12-27      0.157658                 0.214716  0.004495  0.115572   
2019-12-30      0.082726                 0.178728  0.005397  0.129307   
2019-12-31      0.201197                 0.235809  0.004472  0.114913   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.040817                -0.106552  0.021013  0.086292   
2015-01-05     -0.025522                -0.085507  0.027930  0.113983   
2015-01-06     -0.085466                -0.169776  0.025566  0.101196   
2015-01-07      0.004634                -0.042241  0.018500  0.074160   
2015-01-08      0.012798                -0.030820  0.021767  0.088287   
...                  ...                      ...       ...       ...   
2019-12-24     -0.067245                -0.096137  0.004503  0.058347   
2019-12-26      0.049273                -0.042822  0.007601  0.097697   
2019-12-27     -0.011710                -0.070837  0.006995  0.090173   
2019-12-30     -0.126153                -0.122834  0.011030  0.138515   
2019-12-31     -0.040254                -0.083979  0.011169  0.139755   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.106106                -0.335672  0.007238  0.027089   
2015-01-05      0.165348                -0.317678  0.012020  0.045517   
2015-01-06      0.229458                -0.298608  0.021309  0.082789   
2015-01-07      0.263136                -0.289633  0.016825  0.067511   
2015-01-08      0.184825                -0.313211  0.016694  0.067998   
...                  ...                      ...       ...       ...   
2019-12-24      0.127206                -0.331019  0.002436  0.016052   
2019-12-26      0.163725                -0.324720  0.006046  0.040095   
2019-12-27      0.076080                -0.339804  0.008358  0.055157   
2019-12-30      0.088253                -0.337738  0.006108  0.040177   
2019-12-31      0.136944                -0.329335  0.006166  0.040677   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.119599                -0.183746  0.022111  0.027432   
2015-01-05     -0.140884                -0.233109  0.022024  0.026781   
2015-01-06     -0.143580                -0.240928  0.029196  0.033203   
2015-01-07     -0.096138                -0.128050  0.020734  0.025169   
2015-01-08     -0.101294                -0.140758  0.025050  0.029647   
...                  ...                      ...       ...       ...   
2019-12-24     -0.128870                -0.170974  0.003795  0.050399   
2019-12-26      0.265378                -0.013707  0.053140  0.358875   
2019-12-27     -0.106071                -0.161976  0.054885  0.369908   
2019-12-30     -0.219308                -0.205205  0.030696  0.217013   
2019-12-31     -0.106450                -0.162101  0.019446  0.148091   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Empty DataFrame
Columns: [price-change, price-change-percentage, volume, amount, sma10, sma10-FP, macd, macd-SG, macd-histogram, cci-SG, mtm10, roc-SG, roc-FP, rsi, rsi-FP, adosc, adosc-SG, ar26, br26, bias20]
Index: []
Series([], Name: validation, dtype: float64)
            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.058468                -0.081788  0.041580  0.078259   
2015-01-05      0.014167                -0.136340  0.057750  0.107465   
2015-01-06     -0.009830                -0.166750  0.054337  0.099230   
2015-01-07      0.120307                -0.003102  0.048837  0.091057   
2015-01-08      0.113845                -0.012912  0.055954  0.106267   
...                  ...                      ...       ...       ...   
2019-12-24      0.071109                -0.077201  0.011120  0.039318   
2019-12-26      0.095514                -0.061245  0.020222  0.071992   
2019-

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2018-07-03     -0.155839                -0.229946  0.013508  0.003142   
2018-07-05     -0.080235                 0.089254  0.059855  0.013773   
2018-07-06     -0.138315                -0.157865  0.028077  0.006707   
2018-07-09     -0.136563                -0.151061  0.019251  0.004765   
2018-07-10     -0.221179                -0.487076  0.041036  0.009091   
...                  ...                      ...       ...       ...   
2019-12-24     -0.137814                -0.164848  0.002007  0.001722   
2019-12-26     -0.128051                -0.135879  0.022772  0.008095   
2019-12-27     -0.147828                -0.194774  0.019989  0.007239   
2019-12-30     -0.162598                -0.238380  0.030902  0.010462   
2019-12-31     -0.137564                -0.164245  0.015998  0.005993   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.011285                -0.235958  0.013052  0.002196   
2015-01-05      0.024490                -0.170249  0.017628  0.003018   
2015-01-06     -0.028331                -0.266667  0.026601  0.004452   
2015-01-07      0.035054                -0.150435  0.021691  0.003723   
2015-01-08      0.068187                -0.092104  0.029667  0.005346   
...                  ...                      ...       ...       ...   
2019-12-24      0.001921                -0.212182  0.006992  0.004671   
2019-12-26     -0.024249                -0.224333  0.009470  0.006295   
2019-12-27      0.068667                -0.181034  0.011396  0.007673   
2019-12-30     -0.044418                -0.233530  0.015247  0.010177   
2019-12-31      0.062905                -0.183838  0.016689  0.011270   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.003675                 0.060683  0.029356  0.084649   
2015-01-05     -0.061791                -0.000667  0.040888  0.116071   
2015-01-06     -0.064558                -0.004589  0.039932  0.111484   
2015-01-07      0.059976                 0.130103  0.033423  0.094989   
2015-01-08      0.055133                 0.123678  0.045969  0.132777   
...                  ...                      ...       ...       ...   
2019-12-24     -0.005943                 0.059344  0.010248  0.080916   
2019-12-26     -0.008308                 0.058432  0.017087  0.134860   
2019-12-27      0.002729                 0.062688  0.013148  0.103850   
2019-12-30     -0.076100                 0.032307  0.016676  0.130781   
2019-12-31      0.045293                 0.079227  0.020543  0.161905   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.048587                -0.119200  0.006343  0.014077   
2015-01-05      0.004869                -0.217055  0.011941  0.024362   
2015-01-06      0.040427                -0.137853  0.009878  0.020420   
2015-01-07      0.055583                -0.103469  0.007953  0.016835   
2015-01-08      0.089975                -0.025267  0.012019  0.024696   
...                  ...                      ...       ...       ...   
2019-12-24      0.030945                -0.129981  0.000523  0.009446   
2019-12-26      0.033292                -0.127798  0.002789  0.019497   
2019-12-27      0.038579                -0.122791  0.008811  0.046204   
2019-12-30     -0.007209                -0.166628  0.007297  0.039196   
2019-12-31      0.158323                -0.007533  0.006646  0.036788   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.056231                -0.308616  0.018941  0.013508   
2015-01-05      0.058072                -0.301835  0.032002  0.021739   
2015-01-06      0.053503                -0.321219  0.021600  0.014773   
2015-01-07      0.067004                -0.265645  0.020915  0.014311   
2015-01-08      0.081051                -0.206540  0.035561  0.023903   
...                  ...                      ...       ...       ...   
2019-12-24      0.071259                -0.256534  0.008826  0.034308   
2019-12-26      0.087308                -0.242832  0.015576  0.055542   
2019-12-27      0.082418                -0.247061  0.029302  0.098728   
2019-12-30     -0.037369                -0.348572  0.031828  0.103894   
2019-12-31     -0.013398                -0.330097  0.043211  0.135804   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.058161                -0.310465  0.017876  0.020716   
2015-01-05     -0.074109                -0.328852  0.042557  0.048228   
2015-01-06     -0.087711                -0.345239  0.038763  0.043083   
2015-01-07     -0.038930                -0.289070  0.024239  0.026983   
2015-01-08      0.033302                -0.203856  0.026017  0.029659   
...                  ...                      ...       ...       ...   
2019-12-24      0.013134                -0.265255  0.004905  0.018179   
2019-12-26     -0.017824                -0.276744  0.012502  0.045102   
2019-12-27      0.020169                -0.262793  0.012719  0.046154   
2019-12-30     -0.130863                -0.318073  0.019830  0.070678   
2019-12-31     -0.025797                -0.279670  0.015031  0.053825   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.300029                -0.145032  0.026061  0.102596   
2015-01-05     -0.572928                -0.364131  0.047121  0.179270   
2015-01-06     -0.305643                -0.149282  0.072523  0.276291   
2015-01-07     -0.216924                -0.075698  0.030824  0.118996   
2015-01-08     -0.216923                -0.076776  0.035566  0.139112   
...                  ...                      ...       ...       ...   
2019-12-24     -0.323994                -0.162313  0.005708  0.032061   
2019-12-26     -0.313003                -0.156132  0.008869  0.049827   
2019-12-27     -0.329486                -0.165405  0.011966  0.067141   
2019-12-30     -0.376195                -0.191722  0.013564  0.075689   
2019-12-31     -0.297893                -0.147564  0.018077  0.101033   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-09-09     -0.032294                -0.006278  0.183117  0.046663   
2015-09-10     -0.064040                -0.119639  0.206584  0.051339   
2015-09-11     -0.009305                 0.075235  0.148285  0.035967   
2015-09-14     -0.002189                 0.101076  0.122864  0.029462   
2015-09-15     -0.012589                 0.063198  0.201454  0.050609   
...                  ...                      ...       ...       ...   
2019-12-24     -0.009852                 0.062185  0.000000  0.016255   
2019-12-26      0.043787                 0.122424  0.046830  0.057377   
2019-12-27     -0.033388                 0.035956  0.033585  0.045640   
2019-12-30     -0.092502                -0.030072  0.041283  0.051486   
2019-12-31     -0.003284                 0.069639  0.047435  0.056892   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.222948                -0.194709  0.011511  0.030287   
2015-01-05     -0.181129                -0.174159  0.008209  0.023358   
2015-01-06     -0.084945                -0.123499  0.010366  0.027639   
2015-01-07      0.013853                -0.070870  0.006158  0.019746   
2015-01-08      0.031887                -0.062520  0.007758  0.023456   
...                  ...                      ...       ...       ...   
2019-12-24     -0.104025                -0.137032  0.000000  0.004050   
2019-12-26     -0.118923                -0.150655  0.003271  0.007743   
2019-12-27     -0.124673                -0.156097  0.005656  0.010360   
2019-12-30     -0.099320                -0.132819  0.005875  0.010594   
2019-12-31     -0.092263                -0.126269  0.004313  0.008842   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.097154                 0.239237  0.024876  0.068130   
2015-01-05      0.066467                 0.175351  0.036300  0.095946   
2015-01-06      0.056100                 0.153283  0.033166  0.086850   
2015-01-07      0.108350                 0.263516  0.026042  0.070254   
2015-01-08      0.140696                 0.330798  0.026558  0.073641   
...                  ...                      ...       ...       ...   
2019-12-24      0.082913                 0.211602  0.006494  0.082255   
2019-12-26      0.143886                 0.245367  0.011867  0.133965   
2019-12-27      0.097694                 0.219727  0.015647  0.170186   
2019-12-30      0.021016                 0.177674  0.013642  0.149802   
2019-12-31      0.089380                 0.215179  0.015605  0.168439   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.195583                 0.090580  0.016045  0.010759   
2015-01-05      0.184795                 0.075532  0.018253  0.012123   
2015-01-06      0.163735                 0.045712  0.019548  0.012726   
2015-01-07      0.216386                 0.119918  0.013246  0.008672   
2015-01-08      0.254912                 0.174661  0.024408  0.016371   
...                  ...                      ...       ...       ...   
2019-12-24      0.224348                 0.112033  0.007994  0.016841   
2019-12-26      0.252087                 0.124278  0.015051  0.031935   
2019-12-27      0.213818                 0.107321  0.014665  0.031158   
2019-12-30      0.156543                 0.082195  0.017078  0.036021   
2019-12-31      0.220496                 0.110295  0.025140  0.053152   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.168523                -0.116556  0.010300  0.027054   
2015-01-05      0.129800                -0.212658  0.018334  0.047279   
2015-01-06      0.164221                -0.127419  0.026706  0.068732   
2015-01-07      0.157498                -0.144467  0.018938  0.048487   
2015-01-08      0.202139                -0.030954  0.019892  0.051770   
...                  ...                      ...       ...       ...   
2019-12-24      0.109942                -0.156848  0.003204  0.030504   
2019-12-26      0.148912                -0.130467  0.006101  0.057934   
2019-12-27      0.193287                -0.100039  0.006540  0.062303   
2019-12-30      0.149765                -0.129873  0.006836  0.064968   
2019-12-31      0.188735                -0.103176  0.008358  0.079645   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2017-06-14      0.084006                 0.119167  0.033393  0.004780   
2017-06-15     -0.007594                -0.303387  0.025385  0.003481   
2017-06-16      0.075463                 0.078227  0.073908  0.010465   
2017-06-19      0.074988                 0.070161  0.023313  0.003490   
2017-06-20      0.002373                -0.254184  0.024858  0.003587   
...                  ...                      ...       ...       ...   
2019-12-24      0.079734                -0.063440  0.046738  0.034325   
2019-12-26      0.101092                -0.042267  0.089240  0.061933   
2019-12-27      0.012814                -0.130967  0.128164  0.086487   
2019-12-30     -0.231134                -0.375055  0.165580  0.105101   
2019-12-31      0.086379                -0.054981  0.091478  0.060664   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.006864                -0.215334  0.004131  0.013971   
2015-01-05      0.026093                -0.193577  0.002655  0.009034   
2015-01-06      0.049294                -0.167534  0.005305  0.018356   
2015-01-07      0.045532                -0.172372  0.008187  0.028757   
2015-01-08      0.024421                -0.195920  0.005297  0.018697   
...                  ...                      ...       ...       ...   
2019-12-24     -0.042952                -0.226827  0.003252  0.036342   
2019-12-26      0.136487                -0.165478  0.010540  0.119913   
2019-12-27      0.009904                -0.208846  0.004761  0.054142   
2019-12-30     -0.043406                -0.226808  0.004328  0.048809   
2019-12-31      0.082951                -0.184042  0.004324  0.049256   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.118973                 0.210475  0.006922  0.015909   
2015-01-05      0.087454                 0.158119  0.009087  0.020052   
2015-01-06      0.084413                 0.152248  0.013816  0.029128   
2015-01-07      0.126438                 0.223608  0.005416  0.012434   
2015-01-08      0.148004                 0.260228  0.013739  0.029876   
...                  ...                      ...       ...       ...   
2019-12-24      0.122568                 0.206090  0.000774  0.011290   
2019-12-26      0.140263                 0.214799  0.002039  0.020231   
2019-12-27      0.123120                 0.206332  0.001953  0.019658   
2019-12-30      0.066165                 0.178457  0.002659  0.024408   
2019-12-31      0.123397                 0.206502  0.004368  0.036385   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.072440                -0.074621  0.028179  0.039200   
2015-01-05     -0.007268                -0.216608  0.046568  0.062706   
2015-01-06     -0.017706                -0.240294  0.080245  0.103983   
2015-01-07      0.089521                -0.041472  0.064064  0.083801   
2015-01-08      0.139814                 0.053421  0.068087  0.091910   
...                  ...                      ...       ...       ...   
2019-12-24      0.111832                -0.052383  0.006623  0.024599   
2019-12-26      0.049108                -0.094425  0.012073  0.044704   
2019-12-27      0.054248                -0.091026  0.013516  0.049933   
2019-12-30      0.043967                -0.097944  0.014543  0.053517   
2019-12-31      0.079956                -0.073775  0.015815  0.058301   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.078366                -0.252480  0.014362  0.016568   
2015-01-05     -0.108345                -0.284773  0.022246  0.025307   
2015-01-06     -0.118722                -0.296599  0.028372  0.031674   
2015-01-07     -0.124487                -0.303900  0.073558  0.080347   
2015-01-08     -0.054153                -0.225077  0.078425  0.086517   
...                  ...                      ...       ...       ...   
2019-12-24     -0.085637                -0.253395  0.006942  0.024292   
2019-12-26     -0.066484                -0.246598  0.010329  0.036194   
2019-12-27     -0.040147                -0.237263  0.015041  0.052971   
2019-12-30     -0.113171                -0.263092  0.013452  0.047127   
2019-12-31     -0.011416                -0.227071  0.015270  0.053987   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.046779                -0.064708  0.040700  0.024406   
2015-01-05     -0.058282                -0.076620  0.056645  0.033751   
2015-01-06     -0.082056                -0.101470  0.091768  0.053962   
2015-01-07      0.029908                 0.015744  0.100709  0.060314   
2015-01-08      0.065951                 0.051749  0.080515  0.049580   
...                  ...                      ...       ...       ...   
2019-12-24      0.023007                -0.026713  0.012587  0.017481   
2019-12-26     -0.038345                -0.054240  0.034816  0.048336   
2019-12-27     -0.041411                -0.055608  0.024161  0.033518   
2019-12-30     -0.150307                -0.104198  0.036517  0.049959   
2019-12-31      0.009203                -0.032748  0.029826  0.041026   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-05     -0.113752                -0.167456  0.027313  0.001657   
2015-01-06     -0.116016                -0.184155  0.042800  0.002568   
2015-01-07     -0.118280                -0.201298  0.038078  0.002241   
2015-01-08     -0.109225                -0.133378  0.034260  0.002040   
2015-01-09     -0.110357                -0.142167  0.027403  0.001644   
...                  ...                      ...       ...       ...   
2019-12-24      0.009621                -0.106490  0.136690  0.145126   
2019-12-26     -0.102434                -0.154867  0.177084  0.188377   
2019-12-27     -0.163554                -0.180547  0.112538  0.118559   
2019-12-30     -0.187323                -0.190836  0.126592  0.131459   
2019-12-31     -0.074137                -0.142596  0.097438  0.101941   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.063603                -0.194629  0.024910  0.036260   
2015-01-05     -0.115617                -0.298766  0.032414  0.046128   
2015-01-06     -0.131153                -0.333005  0.059211  0.081774   
2015-01-07     -0.025099                -0.113490  0.049657  0.069720   
2015-01-08     -0.024425                -0.113387  0.038719  0.055269   
...                  ...                      ...       ...       ...   
2019-12-24     -0.023746                -0.165765  0.007819  0.033168   
2019-12-26     -0.037729                -0.175522  0.016017  0.068188   
2019-12-27     -0.071524                -0.198717  0.016806  0.071455   
2019-12-30     -0.068416                -0.196595  0.022950  0.097496   
2019-12-31     -0.070358                -0.197930  0.026379  0.111935   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.049691                 0.062831  0.047406  0.017708   
2015-01-05     -0.020174                -0.020699  0.073100  0.026856   
2015-01-06     -0.181093                -0.215384  0.076307  0.026684   
2015-01-07      0.137882                 0.176088  0.050166  0.018115   
2015-01-08      0.227216                 0.282550  0.049243  0.018629   
...                  ...                      ...       ...       ...   
2019-12-24      0.094404                 0.082632  0.017092  0.011300   
2019-12-26     -0.004097                 0.013531  0.017356  0.011420   
2019-12-27     -0.002347                 0.014670  0.018091  0.011834   
2019-12-30     -0.011090                 0.008451  0.016655  0.010884   
2019-12-31      0.009310                 0.022724  0.017079  0.011121   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.063915                -0.472983  0.000812  0.001454   
2015-01-05     -0.060935                -0.466797  0.000579  0.001029   
2015-01-06     -0.048801                -0.440818  0.001296  0.002314   
2015-01-07     -0.055826                -0.455894  0.000698  0.001245   
2015-01-08     -0.022193                -0.383768  0.000992  0.001811   
...                  ...                      ...       ...       ...   
2019-12-24     -0.025937                -0.437246  0.000191  0.001484   
2019-12-26      0.020803                -0.414294  0.001223  0.009624   
2019-12-27      0.006126                -0.421909  0.001156  0.009187   
2019-12-30     -0.108353                -0.477269  0.001150  0.009055   
2019-12-31     -0.075161                -0.461407  0.000650  0.005098   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.048386                 0.001016  0.008278  0.054232   
2015-01-05     -0.125785                -0.052032  0.010599  0.068366   
2015-01-06     -0.094351                -0.030762  0.016095  0.103174   
2015-01-07      0.110068                 0.111984  0.021064  0.142531   
2015-01-08     -0.103247                -0.035981  0.036270  0.243347   
...                  ...                      ...       ...       ...   
2019-12-24     -0.048881                 0.002823  0.003677  0.021419   
2019-12-26     -0.096229                -0.033747  0.006259  0.036184   
2019-12-27     -0.090496                -0.029436  0.005741  0.033002   
2019-12-30     -0.114022                -0.047842  0.007404  0.041981   
2019-12-31     -0.066574                -0.010794  0.006869  0.039042   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.180482                -0.082205  0.010205  0.002946   
2015-01-05      0.171529                -0.135986  0.019354  0.005068   
2015-01-06      0.165986                -0.170573  0.019333  0.004909   
2015-01-07      0.177710                -0.099193  0.032800  0.007989   
2015-01-08      0.194124                 0.004411  0.028672  0.007307   
...                  ...                      ...       ...       ...   
2019-12-24      0.174354                -0.094638  0.012939  0.044743   
2019-12-26      0.191380                -0.086329  0.017715  0.059068   
2019-12-27      0.127478                -0.117465  0.025509  0.081476   
2019-12-30      0.078172                -0.141958  0.025879  0.080981   
2019-12-31      0.244667                -0.059442  0.022942  0.073418   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.230907                 0.536418  0.005768  0.015852   
2015-01-05      0.229712                 0.534969  0.006990  0.019055   
2015-01-06      0.251981                 0.560141  0.014650  0.039960   
2015-01-07      0.289195                 0.602245  0.006423  0.017929   
2015-01-08      0.261247                 0.570241  0.010796  0.030287   
...                  ...                      ...       ...       ...   
2019-12-24      0.223435                 0.545389  0.002184  0.021672   
2019-12-26      0.190255                 0.534899  0.002288  0.022528   
2019-12-27      0.321179                 0.576387  0.003394  0.033575   
2019-12-30      0.173816                 0.529772  0.003555  0.034827   
2019-12-31      0.230907                 0.547701  0.004487  0.043791   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.103619                 0.021651  0.038481  0.122152   
2015-01-05     -0.125202                -0.223014  0.068052  0.209870   
2015-01-06     -0.009323                -0.100906  0.058778  0.179878   
2015-01-07      0.039080                -0.047791  0.058206  0.178374   
2015-01-08      0.072814                -0.010595  0.041548  0.128309   
...                  ...                      ...       ...       ...   
2019-12-24      0.052749                -0.040314  0.015173  0.069327   
2019-12-26      0.031746                -0.055892  0.026557  0.121344   
2019-12-27      0.047497                -0.044240  0.027033  0.123764   
2019-12-30      0.026496                -0.059768  0.028020  0.128196   
2019-12-31      0.068502                -0.028737  0.028562  0.131283   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.026859                -0.073266  0.023351  0.012852   
2015-01-05     -0.033607                -0.079135  0.029190  0.016165   
2015-01-06     -0.146904                -0.174258  0.067761  0.036146   
2015-01-07      0.055183                -0.001372  0.030401  0.016888   
2015-01-08     -0.093984                -0.129753  0.032880  0.017961   
...                  ...                      ...       ...       ...   
2019-12-24     -0.004839                -0.061778  0.015859  0.010819   
2019-12-26     -0.143707                -0.156772  0.031865  0.021111   
2019-12-27     -0.064151                -0.102886  0.039053  0.025757   
2019-12-30     -0.062020                -0.101430  0.022083  0.014508   
2019-12-31     -0.075871                -0.111257  0.028012  0.018251   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.183230                 0.051321  0.025884  0.025936   
2015-01-05      0.131702                -0.007108  0.028448  0.027987   
2015-01-06      0.288267                 0.172485  0.135908  0.137580   
2015-01-07      0.220885                 0.093022  0.080460  0.082769   
2015-01-08      0.373488                 0.259418  0.234098  0.255763   
...                  ...                      ...       ...       ...   
2019-12-24      0.202857                 0.060395  0.028400  0.056999   
2019-12-26      0.260765                 0.093519  0.036109  0.073121   
2019-12-27      0.127793                 0.017744  0.042556  0.085137   
2019-12-30      0.119214                 0.012592  0.048451  0.095706   
2019-12-31      0.211437                 0.065352  0.031757  0.063589   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.221274                -0.175159  0.081638  0.135127   
2015-01-05     -0.281536                -0.216452  0.127264  0.206350   
2015-01-06     -0.229895                -0.182798  0.131011  0.209655   
2015-01-07     -0.149164                -0.127200  0.085316  0.136407   
2015-01-08     -0.000771                -0.023464  0.121599  0.198644   
...                  ...                      ...       ...       ...   
2019-12-24     -0.031777                -0.078401  0.025084  0.070661   
2019-12-26     -0.029182                -0.077803  0.046351  0.131860   
2019-12-27     -0.109537                -0.109818  0.047325  0.135019   
2019-12-30     -0.122495                -0.114932  0.049735  0.142145   
2019-12-31     -0.197658                -0.144341  0.053532  0.152288   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.209909                -0.375160  0.056452  0.030463   
2015-01-05     -0.216257                -0.398780  0.062149  0.032766   
2015-01-06     -0.213083                -0.387455  0.078695  0.040879   
2015-01-07     -0.201444                -0.342857  0.155717  0.082338   
2015-01-08     -0.194038                -0.315219  0.085535  0.046952   
...                  ...                      ...       ...       ...   
2019-12-24     -0.202366                -0.357704  0.008657  0.009831   
2019-12-26     -0.205622                -0.363517  0.015205  0.017316   
2019-12-27     -0.209963                -0.371166  0.015030  0.017068   
2019-12-30     -0.213219                -0.376922  0.019830  0.022362   
2019-12-31     -0.201281                -0.355776  0.019777  0.022492   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.135501                 0.021041  0.049333  0.054964   
2015-01-05     -0.268791                -0.096423  0.078601  0.085162   
2015-01-06     -0.183772                -0.023238  0.098565  0.105446   
2015-01-07      0.007879                 0.151740  0.076461  0.083676   
2015-01-08      0.066962                 0.202182  0.092561  0.104647   
...                  ...                      ...       ...       ...   
2019-12-24     -0.095282                 0.047229  0.025802  0.059956   
2019-12-26      0.010332                 0.091833  0.026683  0.062669   
2019-12-27     -0.146162                 0.025858  0.024315  0.056961   
2019-12-30     -0.181623                 0.010964  0.031675  0.073773   
2019-12-31     -0.149245                 0.024452  0.045809  0.106362   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.295508                -0.196578  0.021605  0.039654   
2015-01-05      0.242966                -0.285404  0.036182  0.060402   
2015-01-06      0.249806                -0.274749  0.037850  0.061905   
2015-01-07      0.282139                -0.219272  0.028419  0.048168   
2015-01-08      0.345251                -0.109506  0.031793  0.054722   
...                  ...                      ...       ...       ...   
2019-12-24      0.249184                -0.240441  0.000235  0.033393   
2019-12-26      0.365149                -0.165674  0.006056  0.057071   
2019-12-27      0.291777                -0.213128  0.007700  0.063714   
2019-12-30      0.167418                -0.292284  0.008123  0.064089   
2019-12-31      0.308254                -0.202351  0.005356  0.053524   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.023063                -0.095279  0.054217  0.029435   
2015-01-05      0.027005                -0.083623  0.099286  0.054750   
2015-01-06     -0.056180                -0.340112  0.116456  0.061108   
2015-01-07      0.034496                -0.057269  0.096708  0.051876   
2015-01-08     -0.009265                -0.197393  0.090296  0.047818   
...                  ...                      ...       ...       ...   
2019-12-24      0.018332                -0.136273  0.040307  0.077642   
2019-12-26     -0.066824                -0.211180  0.033041  0.062614   
2019-12-27      0.000591                -0.151990  0.048699  0.092157   
2019-12-30      0.038439                -0.118122  0.048600  0.092619   
2019-12-31      0.008476                -0.144950  0.041104  0.078360   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.228844                 0.178620  0.030740  0.032746   
2015-01-05      0.264076                 0.250704  0.061071  0.063478   
2015-01-06      0.230820                 0.182723  0.066255  0.068188   
2015-01-07      0.312150                 0.348814  0.099417  0.104948   
2015-01-08      0.290418                 0.300935  0.103008  0.111333   
...                  ...                      ...       ...       ...   
2019-12-24      0.273296                 0.223552  0.009832  0.050568   
2019-12-26      0.314126                 0.243747  0.033611  0.149376   
2019-12-27      0.205795                 0.190184  0.028058  0.125786   
2019-12-30      0.270333                 0.221967  0.021767  0.100250   
2019-12-31      0.271320                 0.222407  0.016491  0.078752   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.222141                -0.102538  0.032084  0.017175   
2015-01-05     -0.216924                -0.082030  0.023776  0.012971   
2015-01-06     -0.220367                -0.095869  0.033232  0.018177   
2015-01-07     -0.203047                -0.029212  0.042570  0.024071   
2015-01-08     -0.205655                -0.041952  0.052198  0.030382   
...                  ...                      ...       ...       ...   
2019-12-24     -0.227775                -0.116609  0.007691  0.021317   
2019-12-26     -0.221306                -0.111452  0.010082  0.027591   
2019-12-27     -0.231636                -0.119682  0.013093  0.035382   
2019-12-30     -0.246139                -0.131269  0.017070  0.045417   
2019-12-31     -0.189587                -0.085927  0.021941  0.058743   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.296939                 0.035780  0.040818  0.030294   
2015-01-05     -0.352596                -0.138295  0.055332  0.038763   
2015-01-06     -0.325777                -0.057033  0.048749  0.033636   
2015-01-07     -0.309450                -0.003352  0.029598  0.020761   
2015-01-08     -0.297095                 0.037539  0.028832  0.020688   
...                  ...                      ...       ...       ...   
2019-12-24     -0.311646                -0.015121  0.005365  0.030345   
2019-12-26     -0.322050                -0.019959  0.010226  0.053863   
2019-12-27     -0.368168                -0.041448  0.014702  0.074761   
2019-12-30     -0.402951                -0.058099  0.012459  0.062879   
2019-12-31     -0.309162                -0.013908  0.010608  0.054197   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2019-12-02      0.280340                 0.369476  0.062551  0.029256   
2019-12-03      0.045185                -0.384842  0.077143  0.031991   
2019-12-04      0.179560                 0.032104  0.086289  0.034986   
2019-12-05      0.126094                -0.158144  0.061600  0.023933   
2019-12-06      0.266619                 0.349968  0.065268  0.026515   
2019-12-09      0.305417                 0.479033  0.076276  0.033018   
2019-12-10      0.110007                -0.197663  0.138877  0.055086   
2019-12-11      0.168677                -0.008383  0.070797  0.028043   
2019-12-12      0.148332                -0.085303  0.065052  0.024779   
2019-12-13      0.236338                 0.239128  0.024598  0.010381   
2019-12-16      0.195174                 0.084478  0.031320  0.012778   
2019-12-17      0.279394                 0.397213  

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.119332                -0.134627  0.076387  0.006752   
2015-01-05     -0.131652                -0.258270  0.086320  0.007290   
2015-01-06     -0.110695                -0.048383  0.101009  0.008554   
2015-01-07     -0.113755                -0.080176  0.046858  0.004040   
2015-01-08     -0.113755                -0.080186  0.054653  0.004681   
...                  ...                      ...       ...       ...   
2019-12-24     -0.100951                -0.010019  0.130486  0.022172   
2019-12-26     -0.101636                -0.014434  0.172893  0.029669   
2019-12-27     -0.114218                -0.079021  0.161578  0.027711   
2019-12-30     -0.144657                -0.233596  0.204996  0.033795   
2019-12-31     -0.105783                -0.034792  0.167168  0.027856   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.002526                -0.277488  0.010509  0.030410   
2015-01-05     -0.066433                -0.384756  0.019508  0.053133   
2015-01-06     -0.085862                -0.420332  0.027633  0.072241   
2015-01-07     -0.005230                -0.282858  0.023019  0.060250   
2015-01-08      0.017899                -0.242226  0.039017  0.101354   
...                  ...                      ...       ...       ...   
2019-12-24     -0.038750                -0.293289  0.002544  0.043003   
2019-12-26      0.133758                -0.179403  0.008784  0.086576   
2019-12-27     -0.050350                -0.300603  0.008508  0.084125   
2019-12-30     -0.100879                -0.333959  0.006538  0.069764   
2019-12-31      0.003097                -0.265770  0.006245  0.067762   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.211527                 0.042076  0.041438  0.099824   
2015-01-05      0.212259                 0.043213  0.095385  0.230177   
2015-01-06      0.182256                -0.003789  0.067386  0.161453   
2015-01-07      0.267872                 0.130978  0.046920  0.114471   
2015-01-08      0.259822                 0.116771  0.037881  0.093860   
...                  ...                      ...       ...       ...   
2019-12-24      0.184062                 0.017774  0.008763  0.045379   
2019-12-26      0.211560                 0.037790  0.017027  0.088244   
2019-12-27      0.241637                 0.059719  0.019875  0.103500   
2019-12-30      0.186638                 0.019684  0.020136  0.104577   
2019-12-31      0.221014                 0.044658  0.021449  0.111618   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.060187                -0.172309  0.045021  0.068286   
2015-01-05      0.139724                -0.110888  0.045517  0.071009   
2015-01-06      0.002229                -0.216235  0.033439  0.051601   
2015-01-07      0.163085                -0.094230  0.069587  0.110751   
2015-01-08      0.079091                -0.159028  0.051105  0.082229   
...                  ...                      ...       ...       ...   
2019-12-24      0.071422                -0.172692  0.010522  0.029368   
2019-12-26     -0.028801                -0.215554  0.025349  0.068984   
2019-12-27      0.062149                -0.176612  0.012812  0.035280   
2019-12-30     -0.028801                -0.215858  0.025021  0.067441   
2019-12-31      0.109407                -0.155901  0.013403  0.036817   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.197564                 0.056190  0.035066  0.023204   
2015-01-05      0.147732                -0.067366  0.038176  0.024757   
2015-01-06      0.155814                -0.048656  0.050386  0.032058   
2015-01-07      0.184545                 0.023803  0.030718  0.019729   
2015-01-08      0.225847                 0.129395  0.041352  0.026991   
...                  ...                      ...       ...       ...   
2019-12-24      0.163092                 0.001427  0.017687  0.030122   
2019-12-26      0.193173                 0.030850  0.021084  0.035748   
2019-12-27      0.170612                 0.008745  0.025048  0.042133   
2019-12-30      0.146546                -0.014911  0.024055  0.040206   
2019-12-31      0.188159                 0.025969  0.024188  0.040459   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2019-05-16     -0.382051                -0.271476  0.363653  0.377611   
2019-05-17      0.202284                 0.136005  0.229947  0.244075   
2019-05-20     -0.123823                -0.093716  0.153075  0.161692   
2019-05-21      0.116638                 0.073261  0.194840  0.208865   
2019-05-22     -0.252290                -0.181324  0.143087  0.151060   
...                  ...                      ...       ...       ...   
2019-12-24     -0.115946                -0.087892  0.080749  0.086262   
2019-12-26      0.025029                 0.008874  0.154296  0.166029   
2019-12-27     -0.117605                -0.088917  0.081933  0.087786   
2019-12-30     -0.173992                -0.127721  0.126709  0.134560   
2019-12-31     -0.067848                -0.054987  0.096420  0.102366   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.168069                -0.047053  0.030072  0.070171   
2015-01-05      0.105933                -0.123783  0.034845  0.079875   
2015-01-06      0.165264                -0.050857  0.032243  0.073660   
2015-01-07      0.215965                 0.013005  0.033729  0.077755   
2015-01-08      0.253506                 0.059489  0.046145  0.108310   
...                  ...                      ...       ...       ...   
2019-12-24      0.231283                -0.005204  0.004591  0.024526   
2019-12-26      0.163107                -0.042115  0.008693  0.046365   
2019-12-27      0.138511                -0.055412  0.009214  0.048931   
2019-12-30      0.185545                -0.029997  0.010689  0.056804   
2019-12-31      0.195254                -0.024738  0.014016  0.074614   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.016143                 0.199153  0.013201  0.009537   
2015-01-05     -0.027297                 0.181420  0.014981  0.010657   
2015-01-06     -0.016730                 0.197486  0.017550  0.012343   
2015-01-07      0.008512                 0.237526  0.017115  0.012056   
2015-01-08      0.043734                 0.293767  0.019490  0.013916   
...                  ...                      ...       ...       ...   
2019-12-24     -0.094218                 0.184753  0.011595  0.027071   
2019-12-26      0.114764                 0.286085  0.015070  0.035440   
2019-12-27      0.037863                 0.248370  0.014898  0.035183   
2019-12-30     -0.083651                 0.190061  0.017178  0.040018   
2019-12-31      0.011447                 0.235678  0.020599  0.047877   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.037168                -0.284377  0.017443  0.045867   
2015-01-05     -0.104260                -0.397193  0.024862  0.064014   
2015-01-06     -0.111220                -0.411948  0.035040  0.088136   
2015-01-07     -0.031930                -0.276104  0.024925  0.062575   
2015-01-08     -0.014422                -0.245305  0.040537  0.102112   
...                  ...                      ...       ...       ...   
2019-12-24     -0.063750                -0.290035  0.004114  0.028161   
2019-12-26      0.095409                -0.185934  0.007992  0.055030   
2019-12-27     -0.086991                -0.304852  0.012486  0.085221   
2019-12-30     -0.139085                -0.338995  0.012637  0.085242   
2019-12-31     -0.019427                -0.261092  0.011557  0.078043   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.261977                 0.086303  0.023558  0.013680   
2015-01-05      0.235596                 0.024675  0.027301  0.015559   
2015-01-06      0.220249                -0.013131  0.039084  0.021656   
2015-01-07      0.336323                 0.265251  0.054644  0.030994   
2015-01-08      0.392442                 0.393719  0.058855  0.035032   
...                  ...                      ...       ...       ...   
2019-12-24      0.300464                 0.131100  0.011993  0.026003   
2019-12-26      0.205136                 0.070124  0.027751  0.058270   
2019-12-27      0.234362                 0.088602  0.025321  0.053018   
2019-12-30      0.190110                 0.059849  0.022851  0.047487   
2019-12-31      0.343895                 0.159808  0.021749  0.045618   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.003325                -0.143449  0.026945  0.069036   
2015-01-05      0.050273                -0.134583  0.031072  0.080021   
2015-01-06     -0.299687                -0.200345  0.044472  0.110109   
2015-01-07      0.216940                -0.101774  0.030994  0.078903   
2015-01-08     -0.082356                -0.159727  0.041722  0.104970   
...                  ...                      ...       ...       ...   
2019-12-24      0.050273                -0.133763  0.007727  0.018325   
2019-12-26     -0.027192                -0.149756  0.015861  0.037226   
2019-12-27     -0.025039                -0.149340  0.010251  0.024036   
2019-12-30      0.012519                -0.141574  0.012941  0.030318   
2019-12-31      0.039906                -0.135900  0.017368  0.040811   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.026509                -0.101607  0.008431  0.057477   
2015-01-05     -0.141931                -0.170605  0.013204  0.088944   
2015-01-06     -0.289210                -0.260255  0.016426  0.107080   
2015-01-07      0.192339                 0.035769  0.011334  0.076468   
2015-01-08     -0.080526                -0.134095  0.014757  0.099204   
...                  ...                      ...       ...       ...   
2019-12-24     -0.091103                -0.130911  0.001983  0.023390   
2019-12-26     -0.076872                -0.126036  0.003554  0.041847   
2019-12-27     -0.073711                -0.124957  0.003511  0.041277   
2019-12-30     -0.120619                -0.141212  0.004707  0.055042   
2019-12-31     -0.012042                -0.103533  0.005658  0.066385   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.203444                 0.149841  0.059335  0.042459   
2015-01-05      0.204420                 0.150977  0.104168  0.074350   
2015-01-06      0.173737                 0.115072  0.081782  0.057927   
2015-01-07      0.242402                 0.195762  0.066116  0.047274   
2015-01-08      0.300355                 0.263193  0.106971  0.078125   
...                  ...                      ...       ...       ...   
2019-12-24      0.201351                 0.151324  0.010923  0.018166   
2019-12-26      0.231296                 0.166675  0.054453  0.089133   
2019-12-27      0.230204                 0.166084  0.025966  0.042827   
2019-12-30      0.143099                 0.121621  0.027425  0.044898   
2019-12-31      0.256882                 0.179841  0.028571  0.047007   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.069083                -0.395936  0.027059  0.034352   
2015-01-05     -0.085007                -0.410478  0.049217  0.061800   
2015-01-06     -0.122159                -0.444642  0.045309  0.056122   
2015-01-07     -0.021316                -0.351529  0.040930  0.051335   
2015-01-08     -0.025032                -0.355432  0.044663  0.056566   
...                  ...                      ...       ...       ...   
2019-12-24     -0.046104                -0.380492  0.009647  0.017957   
2019-12-26     -0.089933                -0.408519  0.020499  0.037233   
2019-12-27     -0.041021                -0.377226  0.023542  0.042853   
2019-12-30     -0.089934                -0.408514  0.026566  0.048068   
2019-12-31     -0.079135                -0.401656  0.020925  0.037919   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.108399                -0.063645  0.013185  0.074555   
2015-01-05      0.121579                -0.041987  0.022537  0.125207   
2015-01-06      0.128743                -0.030133  0.026087  0.144738   
2015-01-07      0.176024                 0.047955  0.019168  0.109234   
2015-01-08      0.159978                 0.020298  0.024564  0.140971   
...                  ...                      ...       ...       ...   
2019-12-24      0.178946                -0.002884  0.003498  0.056745   
2019-12-26      0.135018                -0.031338  0.007263  0.109619   
2019-12-27      0.102524                -0.052152  0.006223  0.094748   
2019-12-30      0.216253                 0.020835  0.010159  0.151755   
2019-12-31      0.118772                -0.041720  0.011374  0.168860   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.290519                -0.159943  0.011490  0.036815   
2015-01-05     -0.519878                -0.244370  0.025142  0.078649   
2015-01-06     -0.511850                -0.242862  0.023490  0.071778   
2015-01-07      0.030963                -0.035243  0.027354  0.088272   
2015-01-08     -0.508409                -0.239747  0.032857  0.103699   
...                  ...                      ...       ...       ...   
2019-12-24     -0.357416                -0.186442  0.006721  0.012727   
2019-12-26     -0.402140                -0.214244  0.015139  0.028317   
2019-12-27     -0.365061                -0.191312  0.019432  0.036220   
2019-12-30     -0.396789                -0.211376  0.017245  0.031786   
2019-12-31     -0.319571                -0.162403  0.019347  0.035929   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2019-06-25     -0.248246                -0.571095  0.061311  0.011553   
2019-06-26     -0.196482                -0.350040  0.041117  0.007975   
2019-06-27     -0.172423                -0.245663  0.047557  0.009669   
2019-06-28     -0.221817                -0.459005  0.051518  0.010247   
2019-07-01     -0.225918                -0.477375  0.090944  0.017413   
...                  ...                      ...       ...       ...   
2019-12-24     -0.205504                -0.386504  0.000949  0.000000   
2019-12-26     -0.199307                -0.351391  0.016599  0.002311   
2019-12-27     -0.215711                -0.443920  0.008400  0.001080   
2019-12-30     -0.206963                -0.394879  0.012333  0.001655   
2019-12-31     -0.196938                -0.338256  0.015912  0.002243   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.004346                -0.169484  0.042401  0.035711   
2015-01-05     -0.057878                -0.248735  0.077999  0.063915   
2015-01-06     -0.102174                -0.309932  0.131511  0.103469   
2015-01-07      0.090830                -0.056467  0.087846  0.070023   
2015-01-08      0.101377                -0.043185  0.086538  0.070078   
...                  ...                      ...       ...       ...   
2019-12-24      0.074461                -0.102209  0.007799  0.014645   
2019-12-26      0.031717                -0.126522  0.015176  0.028456   
2019-12-27      0.042116                -0.120624  0.018548  0.034770   
2019-12-30     -0.014490                -0.152799  0.017113  0.031855   
2019-12-31      0.046737                -0.117988  0.018807  0.035017   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2016-02-17      0.108862                -0.121690  0.065061  0.017702   
2016-02-18     -0.028993                -0.580845  0.056846  0.014573   
2016-02-19      0.051969                -0.316210  0.061316  0.015812   
2016-02-22      0.043764                -0.344502  0.041521  0.010697   
2016-02-23      0.066740                -0.265620  0.054620  0.014327   
...                  ...                      ...       ...       ...   
2019-12-24      0.032275                -0.348659  0.017669  0.024814   
2019-12-26      0.109409                -0.300090  0.037904  0.053751   
2019-12-27      0.013129                -0.360540  0.039476  0.055715   
2019-12-30     -0.053064                -0.402122  0.047118  0.065517   
2019-12-31      0.100109                -0.305601  0.045039  0.063157   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.268302                 0.011135  0.015076  0.016630   
2015-01-05      0.251579                -0.022967  0.015474  0.016876   
2015-01-06      0.283909                 0.042772  0.042681  0.046595   
2015-01-07      0.316611                 0.109893  0.016631  0.018403   
2015-01-08      0.334820                 0.145797  0.023292  0.026298   
...                  ...                      ...       ...       ...   
2019-12-24      0.303604                 0.051175  0.003956  0.014414   
2019-12-26      0.236715                 0.010001  0.014043  0.050904   
2019-12-27      0.359346                 0.085671  0.011625  0.042521   
2019-12-30      0.161650                -0.035884  0.012312  0.044412   
2019-12-31      0.293199                 0.044814  0.017626  0.063673   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.216754                 0.029910  0.023287  0.014173   
2015-01-05      0.195139                -0.002306  0.038685  0.023187   
2015-01-06      0.224536                 0.040942  0.046158  0.027542   
2015-01-07      0.298891                 0.153148  0.037116  0.022661   
2015-01-08      0.300621                 0.153569  0.041128  0.025695   
...                  ...                      ...       ...       ...   
2019-12-24      0.220402                 0.048594  0.006950  0.009618   
2019-12-26      0.277235                 0.085572  0.010492  0.014614   
2019-12-27      0.221417                 0.049291  0.012310  0.017105   
2019-12-30      0.245774                 0.065063  0.014007  0.019491   
2019-12-31      0.224461                 0.051253  0.021390  0.029708   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.000532                 0.196700  0.028494  0.121912   
2015-01-05     -0.045556                 0.152846  0.034467  0.143153   
2015-01-06      0.022978                 0.219279  0.035282  0.146533   
2015-01-07      0.055802                 0.252012  0.021433  0.090569   
2015-01-08      0.114576                 0.309359  0.031814  0.139212   
...                  ...                      ...       ...       ...   
2019-12-24      0.029273                 0.221291  0.006346  0.078586   
2019-12-26      0.158898                 0.265687  0.012362  0.153951   
2019-12-27      0.020098                 0.218164  0.019525  0.241721   
2019-12-30      0.064275                 0.233006  0.019235  0.239579   
2019-12-31      0.074171                 0.236229  0.013398  0.168805   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.107872                 0.407669  0.014863  0.043769   
2015-01-05      0.062521                 0.375669  0.027506  0.080047   
2015-01-06      0.048267                 0.365156  0.027961  0.080258   
2015-01-07      0.249757                 0.510196  0.028578  0.084606   
2015-01-08      0.127956                 0.421752  0.028970  0.086106   
...                  ...                      ...       ...       ...   
2019-12-24      0.160998                 0.436523  0.009849  0.038498   
2019-12-26      0.035309                 0.369371  0.016726  0.064449   
2019-12-27      0.159054                 0.435583  0.028703  0.111381   
2019-12-30      0.151928                 0.431555  0.035362  0.138141   
2019-12-31      0.146097                 0.428303  0.019410  0.076361   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.164664                 0.033767  0.049335  0.012561   
2015-01-05      0.101144                -0.039602  0.048783  0.012240   
2015-01-06      0.128118                -0.008732  0.059822  0.015028   
2015-01-07      0.253418                 0.137336  0.107641  0.028659   
2015-01-08      0.200340                 0.071639  0.095837  0.026207   
...                  ...                      ...       ...       ...   
2019-12-24      0.164664                 0.010092  0.005091  0.004337   
2019-12-26      0.140300                 0.000162  0.016290  0.012571   
2019-12-27      0.119416                -0.008293  0.017294  0.013280   
2019-12-30      0.206431                 0.026986  0.032288  0.024547   
2019-12-31      0.162923                 0.009211  0.023177  0.017873   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.389431                -0.002054  0.027165  0.010929   
2015-01-05      0.337313                -0.134301  0.064981  0.024090   
2015-01-06      0.363226                -0.071249  0.062227  0.022656   
2015-01-07      0.407774                 0.046320  0.039290  0.014721   
2015-01-08      0.458145                 0.179567  0.042913  0.016664   
...                  ...                      ...       ...       ...   
2019-12-24      0.402824                 0.018414  0.010079  0.013065   
2019-12-26      0.439802                 0.055076  0.023675  0.026231   
2019-12-27      0.382152                -0.001988  0.024663  0.027102   
2019-12-30      0.319552                -0.063640  0.028735  0.030454   
2019-12-31      0.417965                 0.033638  0.023483  0.025667   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.163199                -0.090608  0.018523  0.049002   
2015-01-05      0.120902                -0.114561  0.023865  0.061976   
2015-01-06     -0.007226                -0.187725  0.033506  0.083708   
2015-01-07      0.369697                 0.030234  0.035387  0.091155   
2015-01-08      0.314964                -0.004629  0.044963  0.117793   
...                  ...                      ...       ...       ...   
2019-12-24      0.132485                -0.104939  0.004074  0.015692   
2019-12-26      0.126078                -0.107962  0.013056  0.042917   
2019-12-27      0.167079                -0.088923  0.011390  0.037878   
2019-12-30      0.096609                -0.121756  0.012214  0.040052   
2019-12-31      0.209362                -0.069043  0.015754  0.051032   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.003718                 0.000585  0.097885  0.062181   
2015-01-05     -0.067718                -0.083086  0.153587  0.094667   
2015-01-06     -0.062310                -0.079105  0.260107  0.155797   
2015-01-07     -0.049690                -0.064756  0.341110  0.199438   
2015-01-08      0.161239                 0.230661  0.207158  0.127164   
...                  ...                      ...       ...       ...   
2019-12-24      0.021521                 0.032388  0.055523  0.056479   
2019-12-26     -0.008225                 0.008070  0.087479  0.088192   
2019-12-27     -0.077634                -0.049121  0.111137  0.109712   
2019-12-30     -0.054197                -0.031068  0.098752  0.096061   
2019-12-31      0.071099                 0.074663  0.085912  0.084531   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.028396                 0.102272  0.035333  0.054224   
2015-01-05     -0.088889                -0.000012  0.046541  0.070332   
2015-01-06      0.019578                 0.094744  0.073431  0.111498   
2015-01-07      0.066312                 0.135650  0.049298  0.075960   
2015-01-08      0.037214                 0.109519  0.043498  0.067617   
...                  ...                      ...       ...       ...   
2019-12-24     -0.000066                 0.073548  0.007677  0.021740   
2019-12-26     -0.005321                 0.071044  0.020285  0.057064   
2019-12-27      0.023057                 0.084538  0.017811  0.050300   
2019-12-30     -0.026339                 0.061074  0.027169  0.076467   
2019-12-31      0.040924                 0.092996  0.024031  0.068011   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.161977                 0.008944  0.013925  0.003978   
2015-01-05      0.157279                -0.004740  0.017597  0.004927   
2015-01-06      0.147096                -0.034277  0.019432  0.005341   
2015-01-07      0.241867                 0.234871  0.038620  0.010685   
2015-01-08      0.254400                 0.266695  0.033598  0.009557   
...                  ...                      ...       ...       ...   
2019-12-24      0.194514                 0.081314  0.010351  0.013852   
2019-12-26      0.323282                 0.164303  0.017572  0.022656   
2019-12-27      0.130956                 0.040756  0.015565  0.020159   
2019-12-30      0.088849                 0.013641  0.019016  0.024069   
2019-12-31      0.191211                 0.079186  0.020754  0.026133   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02     -0.018086                -0.224550  0.004138  0.032355   
2015-01-05     -0.090344                -0.337119  0.007211  0.053874   
2015-01-06     -0.078074                -0.321792  0.011557  0.083304   
2015-01-07      0.004636                -0.187893  0.009116  0.066246   
2015-01-08      0.039856                -0.129867  0.005084  0.038083   
...                  ...                      ...       ...       ...   
2019-12-24     -0.037335                -0.224923  0.002254  0.052256   
2019-12-26      0.025500                -0.191231  0.001950  0.045758   
2019-12-27     -0.000401                -0.205165  0.003307  0.075922   
2019-12-30     -0.051005                -0.232196  0.003543  0.080681   
2019-12-31      0.016866                -0.195867  0.003111  0.071410   

               sma10  sma10-FP  macd  macd-SG  mac

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            price-change  price-change-percentage    volume    amount  \
Date                                                                    
2015-01-02      0.056542                -0.228037  0.027801  0.026043   
2015-01-05      0.035632                -0.271860  0.048225  0.044624   
2015-01-06      0.029048                -0.286313  0.078369  0.071307   
2015-01-07      0.062157                -0.215798  0.050491  0.046179   
2015-01-08      0.093330                -0.148980  0.062634  0.058770   
...                  ...                      ...       ...       ...   
2019-12-24      0.055045                -0.232178  0.012940  0.048873   
2019-12-26      0.063681                -0.227689  0.020819  0.078750   
2019-12-27      0.028716                -0.245841  0.020357  0.076684   
2019-12-30      0.029981                -0.245238  0.028470  0.106823   
2019-12-31      0.051464                -0.234049  0.030639  0.114906   

               sma10  sma10-FP  macd  macd-SG  mac

In [2]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.feature_selection import mutual_info_regression



def main():
    #全時系列データの呼び出し
    path = '../../stock_data/time_series_data/nasdaq100'
    stock_datas = os.listdir(path)
    print(stock_datas)
    cols = ['price-change', 'price-change-percentage', 'volume', 'amount', 'sma10', 'sma10-FP',
        'macd', 'macd-SG', 'macd-histogram', 'cci-SG', 'mtm10', 'roc-SG', 'roc-FP', 'rsi', 'rsi-FP',
            'adosc', 'adosc-SG', 'ar26', 'br26', 'bias20']
    
    mi_score = pd.DataFrame()
    for i in stock_datas:
        stock_data = pd.read_csv('../../stock_data/time_series_data/nasdaq100/' + i)
        df = stock_data.copy()
        df = df.set_index('Date')
        
        
        feature1(df)
        validation(df)
        df = df['2015-01-01':'2020-01-01']
        
        
        #データセットの用意
        X = df[cols]
        print(X)
        y = df['validation']
        print(y)
        
        
        # All discrete features should now have integer dtypes (double-check this before using MI!)
        discrete_features = X.dtypes == int
        #相互情報量を調査
        try:
            mi_scores = make_mi_scores(X, y, discrete_features)
            print(mi_scores)
            mi_score = pd.concat([mi_score, mi_scores], axis=1)
        except:
            continue
            
    #スコアの平均を求める
    score = mi_score.mean(axis='columns')
    score = score.sort_values(ascending=False)
    print(score)
    
    #グラフで描写する
    plt.figure(dpi=100, figsize=(8, 5))
    plot_mi_scores(score)
    plt.show()
    
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

    
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
#     mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores


        
        
def Zero_One_Scale(df):
    df_scaled = (df - df.min()) / (df.max() - df.min())
    return df_scaled

def One_One_Scale(df):
    df_scaled = 2 * (df - df.min()) / (df.max() - df.min()) - 1
    return df_scaled

def Normalize(df):
    df_normalized = (df - df.mean(axis=0)) / df.std(axis=0)

def RSI(x):
    up, down = [i for i in x if i > 0], [i for i in x if i <= 0]
    if len(down) == 0:
        return 100
    elif len(up) == 0:
        return 0
    else:
        up_average = sum(up) / len(up)
        down_average = - sum(down) / len(down)
        return 100 * up_average / (up_average + down_average)
    
# def SlowK(x):
#     min_price = min(x)
#     max_price = max(x)
#     k = (x[-1] - min_price) / (max_price - min_price)#[-1]でkeyerrorが起こってしまう
#     return k     

    
    
def feature1(df):
    # set return and direction (label)
    df['return'] = np.log(df['Adj Close'] / df['Adj Close'].shift(1))
    df['direction'] = np.where(df['return'] > 0, 1, -1)
    df['direction'] = df['direction'].shift(-1)
    df['return'] = One_One_Scale(df['return'])
    df['return'] = df['return'].shift(-1)
    # feature calculation
    # basic information
    df['price-change'] = df['Adj Close'] - df['Adj Close'].shift(1)
    df['price-change-percentage'] = df['Adj Close'] / df['Adj Close'].shift(1)
    df['volume'] = Zero_One_Scale(df['Volume'])
    df['amount'] = df['Adj Close'] * df['Volume']
    df['amount'] = Zero_One_Scale(df['amount'])
    # simple moving average
    df['sma10'] = df['Adj Close'].rolling(30).mean()
    df['sma10-FP'] = (df['sma10'] - df['sma10'].shift(1)) / df['sma10'].shift(1)
    df['sma10-FP'] = One_One_Scale(df['sma10-FP'])
    df['sma10'] = Zero_One_Scale(df['sma10'])
    # Moving Average Convergence Divergence
    df['macd'] = df['Adj Close'].rolling(12).mean() - df['Adj Close'].rolling(26).mean()
    df['macd-SG'] = df['macd'].rolling(9).mean()
    df['macd-histogram'] = df['macd'] - df['macd-SG']
    df['macd-histogram'] = np.where(df['macd-histogram'] > 0, 1, -1)
    df['macd-SG'] = np.where(df['macd-SG'] > 0, 1, -1)
    df['macd'] = np.where(df['macd'] > 0, 1, -1)
    # Commodity Channel Index in 24 days
    df['typical-price'] = (df['High'] + df['Low'] + df['Close']) / 3
    df['sma-cci'] = df['typical-price'].rolling(24).mean()
    df['mean-deviation'] = np.abs(df['typical-price'] - df['sma-cci'])
    df['mean-deviation'] = df['mean-deviation'].rolling(24).mean()
    df['cci'] = (df['typical-price'] - df['sma-cci']) / (0.015 * df['mean-deviation'])
    df['cci-SG'] = np.where(df['cci'] > 0, 1, -1)
    # MTM 10
    df['mtm10'] = df['Adj Close'] - df['Adj Close'].shift(10)
    df['mtm10'] = np.where(df['mtm10'] > 0, 1, -1)
    # Rate of Change in 10 days
    df['roc'] = (df['Adj Close'] - df['Adj Close'].shift(10)) / df['Adj Close'].shift(10)
    df['roc-SG'] = np.where(df['roc'] > 0, 1, -1)
    df['roc-FP'] = (df['roc'] - df['roc'].shift(1))
    df['roc-FP'] = One_One_Scale(df['roc-FP'])
    # Relative Strength Index in 5 days
    df['rsi'] = df['price-change'].rolling(5).apply(RSI) / 100
    df['rsi-FP'] = (df['rsi'] - df['rsi'].shift(1))
    df['rsi-FP'] = One_One_Scale(df['rsi-FP'])
    # Slow K and Slow D
#     df['slow-k'] = df['Adj Close'].rolling(14).apply(SlowK)
#     df['slow-d'] = df['slow-k'].rolling(14).mean()
#     df['slow-k-FP'] = df['slow-k'] - df['slow-k'].shift(1)
#     df['slow-d-FP'] = df['slow-d'] - df['slow-d'].shift(1)
#     df['slow-k'] = Zero_One_Scale(df['slow-k'])
#     df['slow-d'] = Zero_One_Scale(df['slow-d'])
#     df['slow-k-FP'] = One_One_Scale(df['slow-k-FP'])
#     df['slow-d-FP'] = One_One_Scale(df['slow-d-FP'])
    # ADOSC
    df['adosc'] = ((2 * df['Close'] - df['High'] - df['Low']) / (df['High'] - df['Low'])) * df['Volume']
    df['adosc'] = df['adosc'].cumsum()
    df['adosc-ema3'] = df['adosc'].ewm(span=3, adjust=False).mean()
    df['adosc-ema10'] = df['adosc'].ewm(span=10, adjust=False).mean()
    df['adosc-SG'] = np.where((df['adosc-ema3'] - df['adosc-ema10']) > 0, 1, -1)
    df['adosc'] = Zero_One_Scale(df['adosc'])
    # AR 26
    hp_op = (df['High'] - df['Open']).rolling(26).sum()
    op_lp = (df['Open'] - df['Low']).rolling(26).sum()
    df['ar26'] = hp_op / op_lp
    df['ar26'] = Zero_One_Scale(df['ar26'])
    # BR 26
    hp_cp = (df['High'] - df['Close']).rolling(26).sum()
    cp_lp = (df['Close'] - df['Low']).rolling(26).sum()
    df['br26'] = hp_cp / cp_lp
    df['br26'] = Zero_One_Scale(df['br26'])
    # VR 26
    
    # BIAS 20
    sma20 = df['Adj Close'].rolling(20).mean()
    df['bias20'] = (df['Adj Close'] - sma20) / sma20
    df['bias20'] = np.where(df['bias20'] > 0, 1, -1)
    
    
    
    df['price-change'] = One_One_Scale(df['price-change'])
    df['price-change-percentage'] = One_One_Scale(df['price-change-percentage'])
    # drop row contains NaN
    df.dropna(inplace=True)
    
    # adjust the length of the data, which should be a multiple number of 30.
    length = len(df) // 5
    
    return df[:5 * length]


def validation(df):
    k=10
    df["validation"] = np.nan
    for i in range(0,len(df)-k):
        df["validation"][i] = df['Adj Close'][i+k]/df['Adj Close'][i]
        
    df["validation"] = Zero_One_Scale(df["validation"])

    return df
        

main()